In [9]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
import nltk
import re

import boto3
from botocore.exceptions import ClientError
from io import StringIO

In [10]:
class Processing:
        
    def __init__(self):
        self.message = 'This is init'
        
    def datareading(self):
        bucket1 = 's3://hiringtrendanalysis/jobdesc.csv' # already created on S3
        client = boto3.resource('s3')
        df_jobdesc = pd.read_csv(bucket1)
        df_jobdesc = df_jobdesc.drop('Unnamed: 0', axis=1)
        df_jobdesc.dropna(inplace = True)
        
        bucket2 = 's3://hiringtrendanalysis/techwords.csv'
        client = boto3.resource('s3')
        df_keywords = pd.read_csv(bucket2)
#         df_keywords = df_keywords.drop('Unnamed: 0', axis=1)
        
        return df_jobdesc,df_keywords
    
        #tokenize the sentences into words
    
    def tokenizedWords(self,corpus):
#         unigrams = []
#         bigrams = []
        tokenizedWords = []
        
        stop_words = set(stopwords.words('english'))
            
        #Unigrams
        for para in corpus:
            words = nltk.word_tokenize(para)
            for w in words:
                if w not in stop_words and len(w)>1 and not w.isdigit():
                    tokenizedWords.append(w.lower().strip())
                
        #Bigrams
        for para in corpus:
            words = nltk.word_tokenize(para)
            fword= []
            for w in words:
                if w not in stop_words and len(w)>1 and not w.isdigit():
                    fword.append(w)
#             print(fword)
            
            nList = list(ngrams(fword,2))
            
            for w in nList:
#                 print(w)
                tmp = ''.join(w)
                tokenizedWords.append(tmp.lower().strip())

        return tokenizedWords
    
    def datacleansing(self,df_jobdesc,df_keywords ):
        
        jd_tokens = []
        techwords = []
        
        for index, row in df_keywords.iterrows():
            techwords.append(row['words'])
            
#         print(techwords)
        for index, row in df_jobdesc.iterrows():
            corpus = nltk.sent_tokenize(row['job_desc'])
        
            for i in range(len(corpus)):
                corpus [i] = corpus [i].lower()
                corpus [i] = re.sub(r'\W',' ',corpus [i])
                corpus [i] = re.sub(r'\s+',' ',corpus [i])
            
            jd_tk = self.tokenizedWords(corpus)
            
            for i in jd_tk:
                if i in techwords:
#                     print(i)
                    jd_tokens.append(i)
                else:
                    continue
                    
            row['job_desc'] = jd_tokens
            
            jd_tokens = []
        
        return df_jobdesc 

    
    def wordDictionary(self,df_keywords):
        words_dict = {}
        #create a dictionary for words in corpus with count
        #words_dict = dict(zip(df_keywords.words, count))
        for index, row in df_keywords.iterrows():
            words_dict.update({row['words']:0})
            
        return words_dict
        
    def countWords(self,mydata,df_keywords):
    
        count = 0
        techwords = []
        final = {}
        
        #read final tech words
        for index, row in df_keywords.iterrows():
            techwords.append(row['words'])
        
        
        #code change required
        for index, row in mydata.iterrows():
            words_dict = self.wordDictionary(df_keywords)
            
            for r in row['job_desc']:
                if r in techwords:
#                     print(r,str(words_dict.get(r)))
                    count = words_dict.get(r) + 1
#                     print(count)
                    words_dict.update({r : count})
            
            key = index
            final.update({key:words_dict})
            
#             for i, j in final.items():
#                 print(j)
        
        df_wordcount = pd.DataFrame.from_dict(final, orient = 'index')
        
        df_merged = mydata.merge(df_wordcount, left_index=True, right_index=True)
        
#         df_merged.rename_axis('job_id')

        return df_merged
#             print(final)
    
    def uploadToS3(self,result):
        try:
            bucket = 'hiringtrendanalysis' # already created on S3
            csv_buffer = StringIO()
            result.to_csv(csv_buffer, index = False)
            s3_resource = boto3.resource('s3')
            s3_resource.Object(bucket, 'wordcount.csv').put(Body=csv_buffer.getvalue())
        except ClientError as e:
            logging.error(e)
            return False
        return True
          

In [11]:
if __name__ == '__main__':
    
    proc = Processing()
    df_jobdesc,df_keywords = proc.datareading()
    
    mydata = proc.datacleansing(df_jobdesc,df_keywords)
    
    data = proc.countWords(mydata,df_keywords)
    
    proc.uploadToS3(data)

In [12]:
data.head(25)

,company,location,title,job_desc,role,applicationdevelopment,architecture,artificialintelligence,cloudcomputing,html,...,encryptcontainers,encrypt,configure,configuration,permissions,smoothtransition,traceability,security,monitoring,network
1,Defibtech,"Branford, CT",Software Engineer,"[sql, erp, crossfunctional]",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bart & Associates, Inc.","Linthicum, MD",Mainframe Developer,"[aws, monitoring, aws, aws, aws, configuration...",software engineer,0,0,0,0,0,...,0,0,0,2,0,0,0,1,1,1
3,"Assured Information Security, Inc. (AIS)","Chelmsford, MA",Assistant Software Engineer - Cross Domain and...,"[security, linux]",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,RPS,"South Kingstown, RI",Senior Software Engineer,"[leadership, javascript, leadership, javascrip...",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"CSAA Insurance Group, a AAA Insurer","Glendale, AZ",Junior Software Engineer,"[python, java, javascript, architecture, aws, ...",software engineer,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Snowbound Software Corporation,"Waltham, MA",Jr. Software Engineer,"[algorithms, writing, java, interpersonal]",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Algorand,"Boston, MA",Software Engineer (All Levels),"[security, security, collaboration, network, j...",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
8,"Amazon Dev Center U.S., Inc.","Boston, MA",Software Engineer,"[java, network, linux, security, aws]",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
9,Microsoft,"Austin, TX",Software Engineer,"[java, algorithms]",software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Akkio,"Cambridge, MA",Startup Software Engineer,[python],software engineer,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
